In [19]:
import json

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import bigrams
import itertools


In [2]:
list = []
for line in open('../data/Sports_and_Outdoors_Reviews_training.json', 'r'):
    list.append(json.loads(line))

In [13]:
# create a list, doc_list, with one review per elem
doc_list = []
rating = []
for i in range(0,len(list)):
    if list[i].get('reviewText'): #not all reviews have text
        rating.append(list[i].get('overall'))
        doc_list.append(list[i].get('reviewText'))

In [4]:
def preprocess_data(doc_set):
    """
    Input  : docuemnt list
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : preprocessed text
    """
    # initialize regex tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    # create English stop words list
    en_stop = set(stopwords.words('english'))
    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()
    # list for tokenized documents in loop
    texts = []
    # loop through document list
    for i in doc_set:
        # clean and tokenize document string
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # remove stop words from tokens
        stopped_tokens = [i for i in tokens if not i in en_stop]
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
        # add tokens to list
        texts.append(stemmed_tokens)
    return texts

In [5]:
clean_text=preprocess_data(doc_list)

In [23]:
# copied this from some tutorial, and modified it. Computes bigrams
def generate_co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    vocab_index = {word: i for i, word in enumerate(vocab)}
 
    # Create 15-grams from all words in corpus
    n_grams = list(nltk.everygrams(corpus,min_len=2,max_len=2)) 
    
    m = np.zeros([len(vocab),len(n_grams)]);
    for i,word1 in enumerate(vocab):
        for j,this_n_gram in enumerate(n_grams):
            if word1 in this_n_gram:
                 m[i,j]=1

    co_occurrence_matrix = np.matrix(m)*np.matrix(np.transpose(m))

    wordFreq = np.sum(m,axis=1)
    frqtSrt = np.argsort(wordFreq)
    frqtSrt = frqtSrt[::-1]
    most_common = frqtSrt[0:985]
    
    co_occurrence_matrix = co_occurrence_matrix.take(most_common,axis=1)
    
    # normalize incidence
    #incidence = np.sum(m,axis=1)
    #a,b = np.shape(m)
    #from numpy import matlib as ml
    #incidence = ml.repmat(incidence,n=1,m=a)
    #co_occurrence_matrix = co_occurrence_matrix/(incidence*np.transpose(incidence))
    co_occurrence_matrix = np.log(1 + co_occurrence_matrix)
    from scipy import stats as sts
    co_occurrence_matrix = sts.zscore(sts.zscore(co_occurrence_matrix,axis=0),axis=1)

    vocab_tuple = tuple(vocab_index)
    basis_index = [];
    for i in range(0,len(most_common)):
        basis_index.append(vocab_tuple[most_common[i]])
    
    return co_occurrence_matrix, vocab_index, basis_index

In [28]:
matrix, vocab_index, basis_index = generate_co_occurrence_matrix(clean_data[1:20])

NameError: name 'clean_data' is not defined